In [1]:
from Levenshtein import distance, editops
from keras.models import load_model
from DatasetManager import SignalSequence
from Network import ctc_lambda_func
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
test_files = []
with open('runs/2019-03-11 13:30:15.455773/test.txt') as f:
    test_files = [x[5:].strip() for x in f.readlines()]
test = {'chrM':test_files}
test_len = sum([len(test[x]) for x in test.keys()])
test_seq = SignalSequence(test, number_of_reads=test_len)
model = load_model('runs/2019-03-11 13:30:15.455773/model.h5', custom_objects = {'<lambda>': lambda y_true, y_pred: y_pred})

In [5]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')
plot_model(model.get_layer('model_1'), to_file='model1.png')

In [9]:
from keras.models import Model
sub_model = model.get_layer('model_1')
im_model = Model(inputs=sub_model.get_input_at(0), outputs =sub_model.get_layer('activation_1').output)
preds = im_model.predict_on_batch(test_seq[0][0])

In [118]:
dists = []
ops = []
lens = []
pred_lens = []
real = []
predicted = []
for i in range(len(test_seq)):
    batch = test_seq[i][0]
    preds = im_model.predict_on_batch(batch)
    val = K.ctc_decode(preds, np.full(100, batch['input_length'][0,0]), greedy=False)
    decoded = K.eval(val[0][0])
    for i in range(decoded.shape[0]):
        real_label = batch['the_labels'][i, :batch['label_length'][i,0]]
        real_label = ''.join([str(int(x)) for x in real_label.tolist()])
        pred_label = list(filter(lambda x: x!= -1, decoded[i,:].tolist()))
        pred_label = [str(x) for x in pred_label]
        pred_label = ''.join(pred_label)
        dists.append(distance(pred_label, real_label))
        ops.append(editops(pred_label, real_label))
        lens.append(len(real_label))
        pred_lens.append(len(pred_label))
        real.append(real_label)
        predicted.append(pred_label)

In [119]:
print(sum(dists)/sum(lens))
print(np.mean(lens))
print(np.mean(pred_lens))

0.42003771725649436
65.39888888888889
43.111111111111114


In [124]:
op_counts = {'insert':0, 'replace':0, 'delete':0}
for op in ops:
    for x in op:
        op_counts[x[0]] += 1
for key in op_counts.keys():
    op_counts[key] = op_counts[key] / sum(lens)

In [125]:
op_counts

{'insert': 0.3441784603883858,
 'replace': 0.07247829558776057,
 'delete': 0.00338096128034795}

In [126]:
op_sum = 0
for key in op_counts.keys():
    op_sum +=  op_counts[key]
for key in op_counts.keys():
    op_counts[key] = op_counts[key] / op_sum 
op_counts

{'insert': 0.8193989402580594,
 'replace': 0.17255187477247907,
 'delete': 0.008049184969461635}

In [123]:
import keras.backend as K
import numpy as np

In [47]:
val = K.ctc_decode(preds, np.full(100, 200), greedy=False)

In [48]:
decoded = K.eval(val[0][0])

In [49]:
decoded.shape

(100, 27)

In [73]:
batch = test_seq[0][0]

In [78]:
batch['input_length'][0,0]

200

In [79]:
batch['label_length'][0,0]

29

In [83]:
''.join([str(x) for x in decoded[1,:]])

'132302121323120301301201-1-1-1'

In [87]:
list(filter(lambda x: x!= -1, decoded[1,:].tolist()))

[1, 3, 2, 3, 0, 2, 1, 2, 1, 3, 2, 3, 1, 2, 0, 3, 0, 1, 3, 0, 1, 2, 0, 1]

In [68]:
batch = test_seq[0]

In [69]:
batch[0]

{'the_input': array([[[ 1.406917  ],
         [ 0.06858943],
         [-0.88953143],
         ...,
         [-0.17474285],
         [-1.1632802 ],
         [-0.980781  ]],
 
        [[ 0.6795822 ],
         [ 0.5100046 ],
         [ 0.34042698],
         ...,
         [-0.19656876],
         [ 0.52413607],
         [ 0.31216407]],
 
        [[ 0.43072826],
         [ 0.72029364],
         [ 0.9555655 ],
         ...,
         [ 0.6117066 ],
         [ 0.6117066 ],
         [ 0.15926073]],
 
        ...,
 
        [[-1.1473943 ],
         [ 0.15297389],
         [ 0.64061195],
         ...,
         [ 0.58642995],
         [ 1.2727354 ],
         [ 0.7670366 ]],
 
        [[-1.4859797 ],
         [-1.7050861 ],
         [-1.5882294 ],
         ...,
         [ 1.318582  ],
         [ 0.9387977 ],
         [ 1.0118332 ]],
 
        [[ 0.4327911 ],
         [ 0.34279817],
         [ 0.34279817],
         ...,
         [ 0.942751  ],
         [ 0.40279347],
         [-0.1821605 ]]], dtype=f